In [2]:
import cv2 as cv
import pytesseract

In [3]:
pytesseract.pytesseract.tesseract_cmd = '/usr/local/Cellar/tesseract/5.1.0/bin/tesseract'

In [124]:
def matToText(filename, write=False):
  mat = cv.imread(filename)

  gray = cv.cvtColor(mat, cv.COLOR_BGR2GRAY)
  blured = cv.GaussianBlur(gray, (15, 15), 0, 5)
  th, threshed = cv.threshold(blured, 70, 255, cv.THRESH_BINARY_INV|cv.THRESH_OTSU)
  kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 5))
  threshed = cv.dilate(threshed, kernel)
  hist = cv.reduce(threshed, 1, cv.REDUCE_AVG).reshape(-1)

  kernel = cv.getStructuringElement(cv.MORPH_RECT, (7, 10))
  dilated = cv.dilate(threshed, kernel)
  contours, hier  = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
  rects = [cv.boundingRect(c) for c in contours]
  rects.sort(key=lambda rect: rect[1])

  h,w = mat.shape[:2]
  stack_y = []
  for ii in range(h-1):
    if (hist[ii] and not hist[ii+1]):
      stack_y.append(ii)
    elif (not hist[ii] and hist[ii+1]):
      stack_y.append(ii)

  ocr_text = []
  while(stack_y):
    y1 = stack_y.pop()
    y2 = stack_y.pop()
    line = []
    while rects:
      rect = rects[-1]
      midPoint = rect[1] + rect[3]//2
      if (y2 <=  midPoint <= y1):
        # cv.rectangle(mat, (rect[0],rect[1]), (rect[0]+rect[2],rect[1]+rect[3]), (0, 0, 255), 4);
        line.append(rects.pop())
      else:
        break
    line.sort(key=lambda rect: rect[0]+rect[2]/2)
    ocr_text.append([
      pytesseract.image_to_string(mat[y:y+h, x:x+w], lang='tha+eng')
      for x,y,w,h in line])

  if (write):
    filename = filename[filename.rfind('/')+1:filename.rfind('.')]
    cv.imwrite(f"drawed-{filename}.jpg", mat)
    cv.imwrite(f"thresed-{filename}.jpg", threshed)

  return ocr_text

In [126]:
text = matToText("./bud-images/65075-029.jpg")

print('\n'.join([
  ' '.join([b.replace('\n', '') for b in x]) for x in text[::-1]
  ]))

448
งาน: ปกครอง 8,788,380 บาท
1. งบบุคลากร 6,042,000 บาท
1.1 เงินเดือน 5,186,880 บาท
01101-1 อัตราเดิม  อัตรา 4,581,480 บาท
01102-1 เงินเลือนขั้น 274,920 บาท
01106-1 เงินประจําตําแหน่ง 152,640 บาท
01107-1 เงินค่าตอบแทนรายเดือนของข้าราชการ 177,840 บาท
1.2 ค่าจ้างประจํา 760,800 บาท
01201-1 อัตราเดิม 4 อัตรา 708,060 บาท
01202-1 เงินเพิมค่าจ้างประจํา 42,540 บาท
01205-1 เงินเพิ่มการครองชีพชั่วคราวของลูกจ้างประจํา 8,640 บาท
01206-1 เงินช่วยเหลือค่าครองชีพของลูกจ้างประจํา 1,560 บาท
1.3 ค่าจ้างชั่วคราว 86,400 บาท
02101-1 ค่าจ้างชั่วคราว  อัตรา 67,680 บาท
02102-1 เงินเพิ่มการครองชีพชั่วคราวของลูกจ้างชั่วคราว 4,320 บาท
02103-1 เงินช่วยเหลือค่าครองชีพของลูกจ้างชั่วคราว 14,400 บาท
1.4 ค่าตอบแทน ใช้สอยและวัสดุ 7,920 บาท
03217-1 เงินสมทบกองทุนประกันสังคม 4,320 บาท
03611-1 ค่าเครื่องแต่งกาย 3,600 บาท
2. งบดําเนินงาน 2,330,280 บาท
2.1 ค่าตอบแทน ใช้สอยและวัสดุ 801,480 บาท
 ค่าตอบแทน 232,440 บาท
ค่าอาหารทําการนอกเวลา
2.1.2 ค่าใช้สอย 184,980 บาท
ค่าซ่อมแซมครุภัณฑ์
ค่าซ่อมแซมยานพาหนะ
ค่าบํารุงรักษาซ่อมแซม